## What I do
- make a function that downloads data for all id's from a certain date range
- makes a folder for each subject MID
- creates a plot
- stores self_description.txt
- stores SATs_Grades.txt 

** things to add **
- errors if person doesn't have data

In [71]:
# import 

date = '2017-09-06'
#date = '2017-09-10'
date='2017-09-25'

In [72]:
import MySQLdb
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shutil

%matplotlib inline


In [73]:
db=MySQLdb.connect(host="christest.c5s5wgavsyu2.us-west-2.rds.amazonaws.com",user="christest",
                  passwd="gags1313",db="chrisdb")

In [74]:
# get the header 
db.query("""SELECT column_name FROM information_schema.columns WHERE table_name = 'web_experiments_participant'""")
r=db.store_result() # sends the whole results
header = r.fetch_row(0)
header = [hh[0] for hh in header]
print(header)

['id', 'MID', 'start_date', 'start_time', 'completion_code', 'completed', 'progress', 'total_tasks', 'tasks', 'cb', 'session_id', 'AID', 'HID', 'progress_times', 'cb2', 'progress_times2', 'show_info_sheet', 'agreed_to_be_contacted', 'which_completion_message']


In [75]:
# get MIDs for that day
db.query("""SELECT * FROM web_experiments_participant WHERE (start_date LIKE '"""+'%'+str(date)+'%'+"""')""")
r=db.store_result() # sends the whole results
rows = r.fetch_row(2000) # there shouldn't be more than 2000

MIDs = []
for row in rows:
    MIDs.append(row[1])
MIDs = list(set(MIDs))
MIDs


['102', '101', '14', '6']

In [76]:
MIDs_copy = MIDs[:]
for MID in MIDs:

    failed_mid =False
    
    # create folder for subject
    directory = 'session1/'+str(MID)+'/'
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    for resp in ['work_style_questions','self_describe_page','sat_grades_entry']:
        # get session 1 data for each trial 
        db.query("""SELECT responses,tags FROM web_experiments_beliefupdate_trial WHERE (save_time LIKE '"""+'%'+str(date)+'%'+"""') AND (MID='"""+str(MID)+"""') AND (session ='"""+str(1)+"""') AND (trial_name='"""+resp+"""')""")
        r=db.store_result() # sends the whole results
        rows = r.fetch_row(100)

        if len(rows)==1:
            response_dict = eval(rows[0][0])
                
            # save to json
            with open(directory+resp+'.json', 'w') as fp:
                json.dump(response_dict, fp)
            
            # save tags too (what to call the bars in the plot)
            if resp =='work_style_questions':
                tags = rows[0][1]
                with open(directory+resp+'tags.txt', 'w') as fp:
                    fp.write(tags)

        else:
            # missing something, or has too many rows so fail this subject 
            failed_mid = True

            
    if failed_mid:
        print('Failed on MID:'+MID)
        MIDs_copy.remove(MID) # remove subject from list for making plots 
        shutil.rmtree(directory) # remove folder 
            
MIDs = MIDs_copy
print('remaining MIDs for date:')
print(MIDs)
            
            

remaining MIDs for date:
['102', '101', '14', '6']


In [77]:
tags

"[u'independent worker; group worker', u'advice seeker; problem solver', u'detailed thinker; big picture thinker', u'intuitive thinker; analytic thinker', u'logical presenter; entertaining presenter', u'strategic; adaptive']"

In [78]:
db.close()

In [79]:
for MID in MIDs:
    
    directory = 'session1/'+str(MID)+'/'
    
    # load data
    with open(directory+'work_style_questions.json', 'r') as f:
        ws_dict = json.load(f)
    
    # labels (tags)
    with open(directory+'work_style_questionstags.txt', 'r') as fp:
        tags = fp.read()
    tags = eval(tags) # convert to list 
    tags = [str(tag).split(';') for tag in tags] # convert from unicode and split
    
    # x data (bars)
    x = [round(int(ws_dict['Q'+str(q)])-1.5) for q in range(len(tags))]

    # change to 3 bars 
    x_new = np.zeros((3))
    x_new[0] = (x[0]+x[1])/2.0 + 0.2
    x_new[1] = (x[2]+(4-x[3])+x[4])/3.0 + 0.2
    x_new[2] = x[5] + 0.2

    x = x_new
    print(x)

    tags_new = [['independent worker','group worker'],
                ['analytic thinker','intuitive thinker'],
                ['strategic planner','adaptive planner']]
    tags = tags_new
        
    for ti in range(len(tags)):
        if x[ti]<0:

            #tags[ti][0]='more \n'+tags[ti][0]
            #tags[ti][1]='more \n'+tags[ti][1]
            tags[ti][0]=''+tags[ti][0]
            tags[ti][1]=''+tags[ti][1]
        else:
            #tags[ti][1]='more \n'+tags[ti][1]
            #tags[ti][0]='more \n'+tags[ti][0]
            tags[ti][1]=''+tags[ti][1]
            tags[ti][0]=''+tags[ti][0]

    # place holders for y data 
    y = [str(q) for q in range(len(tags))]
    print(y)

    colors = ['blue' for _y in y]
    
    # plot 
    plt.figure()
    sns.axes_style('white')
    sns.set_style('white')

    ax = sns.barplot(x, y,palette=colors,orient='h',alpha=0.5)
    fig = plt.gcf()
    fig.set_size_inches(4,2)

    for n, (_, _x) in enumerate(zip(y,x)):
        if _x>0:
            weighting1='normal'
            weighting2='bold'
        else:
            weighting1='bold'
            weighting2='normal'
            
        ax.annotate(
            s='{:}'.format(tags[n][0]),
            xy=(-2.5,n),
            ha='right',va='center',
            xytext=(0,0),
            textcoords='offset points',
            weight=weighting1
        )
        
        ax.annotate(
            s='{:}'.format(tags[n][1]), # +'\n (+'+str(abs(_x))+')')
            xy=(2.5,n),
            ha='left',va='center',
            xytext=(0,0),
            textcoords='offset points',
            weight=weighting2
        )

        ax.annotate(
            s='',
            xy=(0,n),
            ha='center',va='center',
            xytext=(10,0),
            textcoords='offset points',
        )  
    # axes formatting
    #ax.axvline(-2.4,c='k',linewidth=0.5)
    #ax.axvline(2.4,c='k',linewidth=0.5)
    ax.axvline(0,c='k',linestyle='--',linewidth=0.5)
    ax.set_xlim([-6,6])
    ax.set_yticks([])
    ax.set_xticks([])
    sns.despine(ax=ax, bottom=True, left=True)
    ax.set_title('Working Style')
    plt.tight_layout()
    plt.savefig(directory+'working_style_profile.png',dpi=100,bbox_inches='tight')
    plt.clf()
    plt.close()

[ 0.7         1.86666667  2.2       ]
['0', '1', '2']
[-0.8  1.2  1.2]
['0', '1', '2']
[ 1.7         1.86666667 -0.8       ]
['0', '1', '2']
[ 0.2        -0.13333333 -0.8       ]
['0', '1', '2']
